## Tutorial of Interventions on Non-transformer Model: MLPs

In [1]:
__author__ = "Zhengxuan Wu"
__version__ = "12/20/2023"

### Overview

This tutorials show how to use this library on non-transformer models, such as MLPs. The set-ups are pretty much the same as standard transformer-based models.

### Set-up

In [2]:
try:
    # This library is our indicator that the required installs
    # need to be done.
    import transformers
    import sys
    sys.path.append("align-transformers/")
except ModuleNotFoundError:
    !git clone https://github.com/frankaging/align-transformers.git
    !pip install -r align-transformers/requirements.txt
    import sys
    sys.path.append("align-transformers/")

In [3]:
import sys
sys.path.append("../..")

import torch
import pandas as pd
from models.basic_utils import embed_to_distrib, top_vals, format_token
from models.configuration_alignable_model import AlignableRepresentationConfig, AlignableConfig
from models.alignable_base import AlignableModel
from models.interventions import VanillaIntervention, RotatedSpaceIntervention
from models.mlp.modelings_mlp import MLPConfig
from models.mlp.modelings_alignable_mlp import create_mlp_classifier

%config InlineBackend.figure_formats = ['svg']
from plotnine import ggplot, geom_tile, aes, facet_wrap, theme, element_text, \
                     geom_bar, geom_hline, scale_y_log10

config, tokenizer, mlp =create_mlp_classifier(MLPConfig(h_dim=32))

[2023-12-20 15:11:01,671] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
loaded model


### Intervene in middle layer by partitioning representations into subspaces

MLP layer may contain only a single "token" representation each layer. As a result, we often want to intervene on a subspace of this "token" representation to localize a concept.

In [4]:
alignable_config = AlignableConfig(
    alignable_model_type=type(mlp),
    alignable_representations=[
        AlignableRepresentationConfig(
            0,
            "block_output",
            "pos",                              # mlp layer creates a single token reprs
            1,
            subspace_partition=[[0,16],[16,32]] # partition into two sets of subspaces
        ),
    ],
    alignable_interventions_type=RotatedSpaceIntervention,
)
alignable = AlignableModel(alignable_config, mlp)

base = {"inputs_embeds": torch.rand(1, 1, 32)}
source = {"inputs_embeds": torch.rand(1, 1, 32)}
print("base", alignable(base))
print("source", alignable(source))

base ((tensor([[[-0.0238,  0.0538]]]),), None)
source ((tensor([[[-0.0288,  0.0543]]]),), None)


In [7]:
_, counterfactual_outputs = alignable(
    base,
    [source],
    {"sources->base": ([[[0]]], [[[0]]])},
    subspaces = [[[0, 1]]]
)

In [8]:
counterfactual_outputs # this should be the same as source.

(tensor([[[-0.0288,  0.0543]]], grad_fn=<UnsafeViewBackward0>),)